In [1]:
import pandas as pd
import numpy as np

# Part 1: Standardization and Handling outliers

In [2]:
path = r'D:\Users\MSI\Desktop\Sparta\pythonweek4'
sales = pd.read_csv(path + '\sales.xls')

In [3]:
sales.head()

,Transaction_date,Product,Price,Payment_Type,Name,City,State,Country,Account_Created,Last_Login,Latitude,Longitude
0,1/2/2009 6:17,Product1,1200,Mastercard,carolina,Basildon,England,United Kingdom,1/2/2009 6:00,1/2/2009 6:08,51.500000,-1.116667
1,1/2/2009 4:53,Product1,1200,Visa,Betina,Parkville,MO,United States,1/2/2009 4:42,1/2/2009 7:49,39.195000,-94.681940
2,1/2/2009 13:08,Product1,1200,Mastercard,Federica e Andrea,Astoria,OR,United States,1/1/2009 16:21,1/3/2009 12:32,46.188060,-123.830000
3,1/3/2009 14:44,Product1,1200,Visa,Gouya,Echuca,Victoria,Australia,9/25/2005 21:13,1/3/2009 14:22,-36.133333,144.750000
4,1/4/2009 12:56,Product2,3600,Visa,Gerd W,Cahaba Heights,AL,United States,11/15/2008 15:47,1/4/2009 12:45,33.520560,-86.802500


In [4]:
# Checking data types
sales.dtypes

Transaction_date     object
Product              object
Price                object
Payment_Type         object
Name                 object
City                 object
State                object
Country              object
Account_Created      object
Last_Login           object
Latitude            float64
Longitude           float64
dtype: object

In [5]:
# 1. Standardize the price column

# We get the standardization values by converting the values to int then using numpy mean and std.
sales.Price = sales.Price.replace(',','', regex=True)
sales['Price'] = sales['Price'].astype('float')

sales.insert(12,'Standardized_price', 0)

meandf = np.mean(sales.Price)
stddf = np.std(sales.Price)

sales.Standardized_price = (sales.Price - meandf)/stddf

sales[['Price', 'Standardized_price']]

,Price,Standardized_price
0,1200.0,-0.372712
1,1200.0,-0.372712
2,1200.0,-0.372712
3,1200.0,-0.372712
4,3600.0,1.610806
...,...,...
995,1200.0,-0.372712
996,3600.0,1.610806
997,7500.0,4.834024
998,1200.0,-0.372712


In [6]:
# 2. Identify and remove outliers in the dataset

#Method 1: Setting upper and lower limit using Mean and 3STD 

upper_limit = meandf + 3*stddf 
lower_limit = max(0, meandf - 3*stddf) 

print(upper_limit, lower_limit)

5280.884178379198 0


In [7]:
sales_cleaned1 = sales[(sales.Price >= lower_limit) & (sales.Price <= upper_limit)]
sales_cleaned1

#29 rows were removed.

,Transaction_date,Product,Price,Payment_Type,Name,City,State,Country,Account_Created,Last_Login,Latitude,Longitude,Standardized_price
0,1/2/2009 6:17,Product1,1200.0,Mastercard,carolina,Basildon,England,United Kingdom,1/2/2009 6:00,1/2/2009 6:08,51.500000,-1.116667,-0.372712
1,1/2/2009 4:53,Product1,1200.0,Visa,Betina,Parkville,MO,United States,1/2/2009 4:42,1/2/2009 7:49,39.195000,-94.681940,-0.372712
2,1/2/2009 13:08,Product1,1200.0,Mastercard,Federica e Andrea,Astoria,OR,United States,1/1/2009 16:21,1/3/2009 12:32,46.188060,-123.830000,-0.372712
3,1/3/2009 14:44,Product1,1200.0,Visa,Gouya,Echuca,Victoria,Australia,9/25/2005 21:13,1/3/2009 14:22,-36.133333,144.750000,-0.372712
4,1/4/2009 12:56,Product2,3600.0,Visa,Gerd W,Cahaba Heights,AL,United States,11/15/2008 15:47,1/4/2009 12:45,33.520560,-86.802500,1.610806
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1/20/2009 10:42,Product2,3600.0,Diners,esther,Huddersfield,NaN,United Kingdom,1/20/2009 9:15,3/1/2009 3:29,53.650000,-1.783333,1.610806
995,1/22/2009 14:25,Product1,1200.0,Visa,Hans-Joerg,Belfast,Northern Ireland,United Kingdom,11/10/2008 12:15,3/1/2009 3:37,54.583333,-5.933333,-0.372712
996,1/28/2009 5:36,Product2,3600.0,Visa,Christiane,Black River,Black River,Mauritius,1/9/2009 8:10,3/1/2009 4:40,-20.360278,57.366111,1.610806
998,1/8/2009 11:55,Product1,1200.0,Diners,julie,Haverhill,England,United Kingdom,11/29/2006 13:31,3/1/2009 7:28,52.083333,0.433333,-0.372712


In [8]:
#Method 2: Using inter-quartile range to set upper limit and lower limit

iqr = np.nanpercentile(sales['Price'],75) - np.nanpercentile(sales['Price'],25)

print(iqr)

#This method will not work as the IQR is zero. This is because most of the values in the dataset are 1200.

0.0


In [9]:
# Using the cleaned data we can now do a better standardization:
meandf = np.mean(sales_cleaned1.Price)
stddf = np.std(sales_cleaned1.Price)

sales_cleaned1.Standardized_price = (sales_cleaned1.Price - meandf)/stddf

sales_cleaned1[['Price', 'Standardized_price']]

D:\anaconda\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Price,Standardized_price
0,1200.0,-0.400430
1,1200.0,-0.400430
2,1200.0,-0.400430
3,1200.0,-0.400430
4,3600.0,2.475457
...,...,...
994,3600.0,2.475457
995,1200.0,-0.400430
996,3600.0,2.475457
998,1200.0,-0.400430


In [10]:
#We can also see the impact of the outliers on the mean and stddev

sales.describe()

,Price,Latitude,Longitude,Standardized_price
count,999.000000,997.000000,999.000000,9.990000e+02
mean,1650.970971,38.984398,-41.417494,-3.737417e-16
std,1210.577115,19.499041,67.377526,1.000501e+00
min,20.000000,-41.465000,-159.485280,-1.347942e+00
25%,1200.000000,35.782500,-88.207775,-3.727122e-01
50%,1200.000000,42.309720,-73.733890,-3.727122e-01
75%,1200.000000,51.050000,4.850000,-3.727122e-01
max,13000.000000,64.837780,174.766667,9.379587e+00


In [11]:
sales_cleaned1.describe()

,Price,Latitude,Longitude,Standardized_price
count,981.000000,979.000000,980.000000,9.810000e+02
mean,1534.169215,39.116890,-41.097999,5.205939e-18
std,834.950840,19.433659,67.592075,1.000510e+00
min,20.000000,-41.465000,-159.485280,-1.814408e+00
25%,1200.000000,36.013475,-87.861807,-4.004304e-01
50%,1200.000000,42.395830,-73.722083,-4.004304e-01
75%,1200.000000,51.083333,4.916667,-4.004304e-01
max,3600.000000,64.837780,174.766667,2.475457e+00


In [12]:
# The max and high values were removed. Lastly we export the output csv file.

sales_cleaned1.to_csv(path + '\Rangel_sales.csv')

# Part 2: Handling missing values

In [13]:
df = pd.read_csv(path + '\Handling_Missing_Values.xls')
df.head()

,ID,Sex,Age,Income,Employed,Children,Buy_Car
0,1,Male,25,25146.0,Single,0.0,No
1,2,Male,30,26939.0,Married,2.0,Yes
2,3,Male,27,26693.0,Married,0.0,No
3,4,Male,28,26666.0,Married,3.0,Yes
4,5,Male,29,25899.0,Married,0.0,No


In [14]:
# Checking which columns/attributes have null values
df.isnull().sum()

ID          0
Sex         4
Age         0
Income      3
Employed    0
Children    2
Buy_Car     0
dtype: int64

In [15]:
# Sex
df[df.Sex.isnull()]

,ID,Sex,Age,Income,Employed,Children,Buy_Car
7,8,NaN,30,26037.0,Married,2.0,Yes
9,10,NaN,28,NaN,Single,NaN,No
14,15,NaN,28,26234.0,Married,0.0,No
17,18,NaN,28,NaN,Single,0.0,Yes


#### Here, it is best to just preserve the data as is, since we can still get useful information from these. I believe that setting it as not specified/NA will not impact analysis, unless the analysis is focused on sex categorization/demographics.

In [16]:
df['Sex'] = df.Sex.fillna('Not Specified')
df

,ID,Sex,Age,Income,Employed,Children,Buy_Car
0,1,Male,25,25146.0,Single,0.0,No
1,2,Male,30,26939.0,Married,2.0,Yes
2,3,Male,27,26693.0,Married,0.0,No
3,4,Male,28,26666.0,Married,3.0,Yes
4,5,Male,29,25899.0,Married,0.0,No
5,6,Male,28,26462.0,Married,1.0,No
6,7,Female,28,NaN,Married,3.0,Yes
7,8,Not Specified,30,26037.0,Married,2.0,Yes
8,9,Female,28,26167.0,Married,1.0,Yes
9,10,Not Specified,28,NaN,Single,NaN,No


In [17]:
# Income
df[df.Income.isnull()]

,ID,Sex,Age,Income,Employed,Children,Buy_Car
6,7,Female,28,NaN,Married,3.0,Yes
9,10,Not Specified,28,NaN,Single,NaN,No
17,18,Not Specified,28,NaN,Single,0.0,Yes


#### In this case, most of the income values are close to each other, thus I think it is valid to fill the null values with the mean of the non null values.

In [18]:
df['Income'] = df.Income.fillna(np.mean(df.Income))
df

,ID,Sex,Age,Income,Employed,Children,Buy_Car
0,1,Male,25,25146.000000,Single,0.0,No
1,2,Male,30,26939.000000,Married,2.0,Yes
2,3,Male,27,26693.000000,Married,0.0,No
3,4,Male,28,26666.000000,Married,3.0,Yes
4,5,Male,29,25899.000000,Married,0.0,No
5,6,Male,28,26462.000000,Married,1.0,No
6,7,Female,28,26245.705882,Married,3.0,Yes
7,8,Not Specified,30,26037.000000,Married,2.0,Yes
8,9,Female,28,26167.000000,Married,1.0,Yes
9,10,Not Specified,28,26245.705882,Single,NaN,No


In [19]:
#Children
df[df.Children.isnull()]

,ID,Sex,Age,Income,Employed,Children,Buy_Car
9,10,Not Specified,28,26245.705882,Single,NaN,No
11,12,Male,30,26195.000000,Single,NaN,Yes


#### We know in real life that the number of children is a big factor in whether a person is capable of buying a car. For ease of analysis, we can just drop these rows, or we could set the number to the mode of the data.

In [20]:
df['Children'] = df.Children.fillna(df['Children'].mode()[0])
df

,ID,Sex,Age,Income,Employed,Children,Buy_Car
0,1,Male,25,25146.000000,Single,0.0,No
1,2,Male,30,26939.000000,Married,2.0,Yes
2,3,Male,27,26693.000000,Married,0.0,No
3,4,Male,28,26666.000000,Married,3.0,Yes
4,5,Male,29,25899.000000,Married,0.0,No
5,6,Male,28,26462.000000,Married,1.0,No
6,7,Female,28,26245.705882,Married,3.0,Yes
7,8,Not Specified,30,26037.000000,Married,2.0,Yes
8,9,Female,28,26167.000000,Married,1.0,Yes
9,10,Not Specified,28,26245.705882,Single,0.0,No


In [21]:
# Lastly we export the csv

df.to_csv(path + '\Rangel_empty.csv')

#### For the last part (Application), I am including two ipynb files. The first file  'Add_columns_landslide_butuan.ipynb' is sample of me categorizing the intensity and magnitude of possible Earthquake Induced Landslides (EIL) in Butuan from a landslide susceptibility dataset compiled by Phivolcs. 

#### The other ipynb file 'Aggregate.ipynb' is me compiling different geographical and demographic csv files into one (settlement data, night time lights, NDVI(vegetation index), landslide susceptibility, population data).  They are grouped by location, with barangay level granularity. I cleaned and fixed the missing values/outliers of some of these datasets before aggregating them into one. I used these for a project before, but I am sure the data is now outdated. Thank you!